# Лабораторная работа 2: Embedding и анализ семантической близости

**Цель:**  
- Установить и настроить Ollama.  
- Выбрать модель для генерации эмбеддингов.  
- Исследовать, как семантически близкие предложения отображаются в векторном пространстве.

**Задачи:**
- Установка Ollama (готовый compose).
- Генерация эмбеддингов для набора из минимум 50 предложений выбранной предметной области.
- Расчёт косинусного сходства для 20 пар предложений, известных своей семантической близостью.

**Метрики:**
- Среднее косинусное сходство для схожих по смыслу предложений должно быть не ниже 0.8.
- Визуализация матрицы сходства (например, heatmap с использованием matplotlib/plotly).

**Необходимые инструменты:**
- **Ollama**  
- **ollama-langchain** ([документация](https://python.langchain.com/api_reference/ollama/embeddings/langchain_ollama.embeddings.OllamaEmbeddings.html#langchain_ollama.embeddings.OllamaEmbeddings))
- **scikit-learn** для вычисления косинусного сходства.
- **matplotlib/plotly** для визуализации.

Импорт библиотек и настройка модели

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama.embeddings import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="nomic-embed-text")

## Подготовка данных

В данном примере используется небольшой набор из 4 предложений для демонстрации работы, однако для выполнения лабораторного задания необходимо использовать не менее 50 предложений выбранной предметной области.

In [2]:
# Пример небольшого набора предложений
sentences = [
    "Привет, как дела?",
    "Здравствуйте, как вы поживаете?",
    "Сегодня прекрасная погода.",
    "Я люблю программировать на Python."
]

# Для лабораторной работы: дополните список не менее чем 50 предложениями.

## Генерация эмбеддингов и расчёт косинусного сходства

На данном этапе генерируются эмбеддинги для каждого предложения с использованием модели Ollama. Затем с помощью метрики косинусного сходства вычисляется матрица, отображающая степень семантической близости между всеми парами предложений.

In [3]:
# Генерация эмбеддингов для предложений
embeddings = embeddings_model.embed_documents(sentences)

# Вычисление матрицы косинусного сходства
similarity_matrix = cosine_similarity(embeddings)

## Визуализация матрицы косинусного сходства

Для наглядного представления результатов используется тепловая карта (heatmap) с помощью библиотеки Plotly. Рекомендуется дополнительно проверить, что для выбранных 20 пар предложений, известных своей семантической близостью, значения косинусного сходства составляют не менее 0.8.

In [4]:
# Создание аннотированной тепловой карты
fig = px.imshow(
    similarity_matrix,
    x=sentences,
    y=sentences,
    color_continuous_scale='Viridis',
    aspect='auto',
    text_auto='.2f'
)

# Обновление макета графика
fig.update_layout(
    title='Матрица косинусного сходства',
    xaxis_title='Предложения',
    yaxis_title='Предложения'
)

# Отображение графика
fig.show()

## Автоматический подбор и визуализация топ-20 пар с высокой семантической близостью

В этой ячейке производится автоматический отбор всех уникальных пар предложений, у которых косинусное сходство не ниже порогового значения (0.8). Затем отобранные пары сортируются по убыванию сходства и выводятся в виде таблицы с тремя столбцами:
- **Предложение 1**
- **Предложение 2**
- **Сходство**

In [6]:
# Устанавливаем пороговое значение сходства
threshold = 0.8
n = len(sentences)
similar_pairs = []

# Перебираем все уникальные пары (i, j) с i < j
for i in range(n):
    for j in range(i + 1, n):
        sim = similarity_matrix[i, j]
        if sim >= threshold:
            similar_pairs.append((sentences[i], sentences[j], sim))

# Преобразуем список пар в DataFrame и сортируем по убыванию сходства
df_pairs = pd.DataFrame(similar_pairs, columns=['Предложение 1', 'Предложение 2', 'Сходство'])
df_pairs = df_pairs.sort_values(by='Сходство', ascending=False).head(20)

# Выводим таблицу с результатами
df_pairs

Предложение 1                    Предложение 2  Сходство
0  Привет, как дела?  Здравствуйте, как вы поживаете?  0.827811

## Дополнительные исследования и эксперименты
- Влияние порога сходства: Изменение порогового значения (например, 0.7 или 0.9) для отбора пар и анализ полученных результатов.
- Сравнение моделей эмбеддингов: Попробуйте использовать различные модели (если доступны) и сравните, как меняется качество представленных эмбеддингов.
- Анализ кластеризации: Используйте алгоритмы кластеризации (например, K-means) для группировки предложений по сходству и визуализируйте полученные кластеры.